In [1]:
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/AI&Ethics/'
os.chdir(root_path)

Mounted at /content/gdrive


In [ ]:
!pip install datasets
!pip install sentencepiece==0.1.94
# !pip install transformers==4.0.1
!pip install demoji
!pip install tweet-preprocessor
!pip install transformers
!pip install ekphrasis

In [2]:
import numpy as np
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer
import json
import pickle
import torch.nn as nn
import torch
import copy
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score 
from tqdm import tqdm
import demoji 
import random
demoji.download_codes() 
import preprocessor as p
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
model_path = 'ai4bharat/indic-bert'
model_path = 'bert-base-uncased'
annotator_file = "data/annotators_demography.csv"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    fix_html=True,  # fix HTML tokens
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    #corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

... OK (Got response in 0.60 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


In [ ]:
# !wget http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip glove.840B.300d.zip -d Data

In [6]:
# !python convert_to_word2vec.py

tcmalloc: large alloc 2635227136 bytes == 0x5616261ba000 @  0x7f48ce89a001 0x7f48cb32454f 0x7f48cb374b58 0x7f48cb378b17 0x7f48cb417203 0x5616220af0e4 0x5616220aede0 0x5616221236f5 0x56162211de0d 0x5616220b077a 0x56162211f86a 0x56162211db0e 0x5616220b077a 0x56162211f86a 0x56162211db0e 0x56162211d813 0x5616221e7592 0x5616221e790d 0x5616221e77b6 0x5616221bf103 0x5616221bedac 0x7f48cd682bf7 0x5616221bec8a


# Run these

In [3]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)

In [4]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

In [5]:
class Dataset():
    def __init__(self, data, batch_size = 32):
        self.data = data
        # self.val_data = val_data
        self.batch_size = batch_size

        self.label_dict = {0: 0,
                            1: 1,
                            2: 2}
                                    
        self.count_dic = {}
        self.inputs, self.labels = self.process_data(self.data)
        self.DataLoader = self.get_dataloader(self.inputs, self.labels)
        # self.train_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'train')
        # self.val_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')
        # self.test_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')

    def ek_extra_preprocess(self, text):
        remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
        word_list=text_processor.pre_process_doc(text)
        word_list=list(filter(lambda a: a not in remove_words, word_list)) 
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        return sent

    def tokenize(self, sentences, padding = True, max_len = 128):
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        input_ids, attention_masks, token_type_ids = [], [], []
        for sent in sentences:
            encoded_dict = tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len, 
                                                    padding='max_length', 
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt', 
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
        
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}
    
    def process_data(self, data):
        sentences, labels = [], []
        print(len(data))
        for label, sentence in zip(list(data['label']), list(data['text'])):
            label = self.label_dict[label]
            sentence = self.ek_extra_preprocess(sentence)
            sentences.append(sentence)
            labels.append(label)
        inputs = self.tokenize(sentences)
        return inputs, torch.Tensor(labels)
    
    def get_dataloader(self, inputs, labels, train = True):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'], labels)
        if train:
            sampler = RandomSampler(data)
        else:
            sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=self.batch_size)

In [6]:
from keras.preprocessing.sequence import pad_sequences
def ek_extra_preprocess(text):
        remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
        word_list=text_processor.pre_process_doc(text)
        word_list=list(filter(lambda a: a not in remove_words, word_list)) 
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        return sent

def encodeData(dataframe,vocab):
    tuple_new_data=[]
    for index,row in tqdm(dataframe.iterrows(),total=len(dataframe)):   
        list_token_id=[]
        text = row['text'][1:-1].replace(',', '').replace('\'', '')
        for word in ek_extra_preprocess(text).split():
            # print(word)
            try:
                index=vocab.stoi[word]
            except KeyError:
                index=vocab.stoi['unk']
            list_token_id.append(index)
        tuple_new_data.append((list_token_id,row['label']))
    return tuple_new_data

def return_dataloader(input_ids,labels,is_train=False):
    inputs = torch.tensor(input_ids)
    labels = torch.tensor(labels,dtype=torch.long)
    data = TensorDataset(inputs,labels)
    if(is_train==False):
        sampler = SequentialSampler(data)
    else:
        sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=32)
    return dataloader

def combine_features(tuple_data, is_train=False):
    input_ids =  [ele[0] for ele in tuple_data]
    labels = [ele [1] for ele in tuple_data]
    
    input_ids = pad_sequences(input_ids,maxlen=128, dtype="long", 
                          value=0, truncating="post", padding="post")

    dataloader=return_dataloader(input_ids,labels,is_train)
    return dataloader

In [47]:
import copy
def flat_accuracy(preds, labels):
    
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
    except:
        preds = preds.reshape(1, -1)
        pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
 
def get_predicted(preds):
    
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
    except:
        preds = preds.reshape(1, -1)
        pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
 
def evaluate(test_dataloader, model):
    model.eval()
    y_preds, y_test = np.array([]), np.array([])

    for batch in test_dataloader:
        b_input_ids, b_labels = batch[0].to(device), batch[1].to(device).long()
        with torch.no_grad():        
            ypred = model(b_input_ids)
        ypred = ypred.cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        y_preds = np.hstack((y_preds, get_predicted(ypred)))
        y_test = np.hstack((y_test, label_ids))

    macro_f1 = f1_score(y_test, y_preds, average='macro')
    report = classification_report(y_test, y_preds)
    print(report)
    return macro_f1, y_preds, y_test
 
def train(training_dataloader, validation_dataloader, model, filepath = None, weights = None, learning_rate = 2e-5, epochs = 1, print_every = 10):
    total_steps = len(training_dataloader) * epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # scheduler = get_linear_schedule_with_warmup(optimizer, 
    #                                             num_warmup_steps = 0, # Default value in run_glue.py
    #                                             num_training_steps = total_steps)
    
    best_weighted_f1 = 0
    best_model = None
    # current_epoch, best_weighted_f1 = load_metrics(filepath, model, optimizer)
    if weights == None:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.CrossEntropyLoss(weight=weights)
    for epoch_i in tqdm(range(0, epochs)):
        model.train()
        for step, batch in enumerate(training_dataloader):
            b_input_ids, b_labels = batch[0].to(device), batch[1].to(device).long()
            
            outputs = model(b_input_ids)
            try:
                loss = criterion(outputs, b_labels)
            except:
                print('WTF')
                continue
 
            if step%print_every == 0:
                print(loss.item())
            model.zero_grad()
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            # scheduler.step()
 
        print('### Validation Set Stats')
        weighted_f1, ypred, ytest = evaluate(validation_dataloader, model)
        print("  Macro F1: {0:.3f}".format(weighted_f1))
        if weighted_f1 > best_weighted_f1:
            print(best_weighted_f1)
            print(weighted_f1)
            best_weighted_f1 = weighted_f1
            best_model = copy.deepcopy(model)
            # save_metrics(filepath, epoch_i, model, optimizer, weighted_f1)
        
    return best_model

In [16]:
def global_max_pooling(tensor, dim, topk):
    """Global max pooling"""
    ret, _ = torch.topk(tensor, topk, dim)
    return ret
    
class CNN_GRU(nn.Module):
    def __init__(self, embeddings):
        super(CNN_GRU, self).__init__()
        self.dropout_embed = nn.Dropout2d(0.2)
        self.dropout_fc = nn.Dropout(0.1)
        self.embedding = nn.Embedding(embeddings.shape[0], 300)
        self.embedsize = 300
        self.embedding.weight = nn.Parameter(torch.tensor(embeddings.astype(np.float32), dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.conv1 = nn.Conv1d(self.embedsize,100, 2)
        self.conv2 = nn.Conv1d(self.embedsize,100, 3,padding=1)
        self.conv3 = nn.Conv1d(self.embedsize,100, 4,padding=2)
        self.maxpool1D = nn.MaxPool1d(4, stride=4)
        self.seq_model = nn.GRU(100, 100, bidirectional=False, batch_first=True,dropout=0.1)
        self.out = nn.Linear(100, 3)
        
    def forward(self,input_ids=None,attention_mask=None,attention_vals=None,labels=None,device=None):
        batch_size=input_ids.size(0)
        h_embedding = self.embedding(input_ids)
        h_embedding = self.dropout_embed(h_embedding)
        new_conv1=self.maxpool1D(self.conv1(h_embedding.permute(0,2,1)))
        new_conv2=self.maxpool1D(self.conv2(h_embedding.permute(0,2,1)))
        new_conv3=self.maxpool1D(self.conv3(h_embedding.permute(0,2,1)))
        concat=self.maxpool1D(torch.cat([new_conv1, new_conv2,new_conv3], dim=2))
        h_seq, _ = self.seq_model(concat.permute(0,2,1))
        global_h_seq=torch.squeeze(global_max_pooling(h_seq, 1, 1)) 
        global_h_seq = self.dropout_fc(global_h_seq)
        output=self.out(global_h_seq)
        return output

In [9]:
class Vocab_own():
    def __init__(self,dataframe, model):
        self.itos={}
        self.stoi={}
        self.vocab={}
        self.embeddings=[]
        self.dataframe=dataframe
        self.model=model
    
    ### load embedding given a word and unk if word not in vocab
    ### input: word
    ### output: embedding,word or embedding for unk, unk
    def load_embeddings(self,word):
        try:
            return self.model[word],word
        except KeyError:
            return self.model['unk'],'unk'
    
    ### create vocab,stoi,itos,embedding_matrix
    ### input: **aself
    ### output: updates class members
    def create_vocab(self):
        count=1
        for text in list(self.dataframe['text']):
            text = text[1:-1].replace(',', '').replace('\'', '')
            for word in text.split():
                # print(word)
                # break
                vector,word=self.load_embeddings(word)      
                try:
                    self.vocab[word]+=1
                except KeyError:
                    if(word=='unk'):
                        print(word)
                    self.vocab[word]=1
                    self.stoi[word]=count
                    self.itos[count]=word
                    self.embeddings.append(vector)
                    count+=1
        self.vocab['<pad>']=1
        self.stoi['<pad>']=0
        self.itos[0]='<pad>'
        self.embeddings.append(np.zeros((300,), dtype=float))
        self.embeddings=np.array(self.embeddings)
        print(self.embeddings.shape)

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
wordvec_model = KeyedVectors.load_word2vec_format('Data/glove.840B.300d_w2v.txt', binary=False)
# len(df_train), len(df_val), len(df_test)

In [13]:
wordvec_model

In [34]:
class_names = ['Caucasian', 'Hispanic', 'Indian', 'African']
parameter = 'Race'
# class_names = ['USA', 'Brazil', 'India', 'Spain', 'Canada', 'Italy']
# parameter = 'Country'
# class_names = ['Christian','Other', 'Nonreligious', 'Hindu']
# parameter = 'Religion'
# class_names = ['Male','Female', 'Prefer not to disclose']
# parameter = 'Gender'
df = pd.read_csv('data/annotators_race.csv')
# df = pd.read_csv('data/annotators_final.csv')
# df = pd.read_csv('data/annotators_religion.csv')
# df = pd.read_csv('data/annotators_gender.csv')
# print(df['Gender'])
df = df[df[parameter].isin(class_names)]
df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname=parameter,
                                         frac_train=0.8, frac_val=0.1, frac_test=0.1,
                                         random_state=42)
# len(df_train), len(df_val), len(df_test)

try:
    with open('results_individual_cnn.pkl', 'rb') as f:
        results = pickle.load(f)
except:
    results = {}
    with open('results_individual_cnn.pkl', 'wb') as f:
        pickle.dump(results, f)
print(results)
if parameter not in results: results[parameter] = {}

{'Gender': {'Male': {'Male': {'f1': 0.5664444981700915, 'acc': 0.5748148148148148}, 'Female': {'f1': 0.6015680640032799, 'acc': 0.6103703703703703}, 'Prefer not to disclose': {'f1': 0.5679012345679012, 'acc': 0.72}}, 'Female': {'Male': {'f1': 0.5362635930873715, 'acc': 0.5392592592592592}, 'Female': {'f1': 0.6462955946778027, 'acc': 0.6503703703703704}, 'Prefer not to disclose': {'f1': 0.5068480725623583, 'acc': 0.6}}, 'Prefer not to disclose': {'Male': {'f1': 0.38345132472898275, 'acc': 0.46370370370370373}, 'Female': {'f1': 0.435197883041137, 'acc': 0.5096296296296297}, 'Prefer not to disclose': {'f1': 0.42619047619047623, 'acc': 0.76}}}, 'Religion': {'Christian': {'Christian': {'f1': 0.5708709589396151, 'acc': 0.5879629629629629}, 'Other': {'f1': 0.542512077294686, 'acc': 0.631578947368421}, 'Nonreligious': {'f1': 0.5614652301339088, 'acc': 0.568039950062422}, 'Hindu': {'f1': 0.5587666135034556, 'acc': 0.646551724137931}}, 'Other': {'Christian': {'f1': 0.5610046850598646, 'acc': 0.5

In [35]:
for c1 in class_names:
    if c1 not in results[parameter]: results[parameter][c1] = {}
    train_df = df_train[df_train[parameter] == c1]
    val_df = df_val[df_val[parameter] == c1]

    vocab_own = Vocab_own(train_df, wordvec_model)
    vocab_own.create_vocab()

    X_train=encodeData(train_df,vocab_own)
    train_dataloader = combine_features(X_train,True)

    X_val=encodeData(val_df,vocab_own)
    val_dataloader = combine_features(X_val)

    model = CNN_GRU(vocab_own.embeddings).to(device)
    model = train(train_dataloader, val_dataloader, model, epochs = 20)

    for c2 in class_names:
        test_df = df_test[df_test[parameter] == c2]
        X_test=encodeData(test_df,vocab_own)
        test_dataloader = combine_features(X_test)
        f1, ypreds, ytest = evaluate(test_dataloader, model)
        acc = accuracy_score(ytest, ypreds)
        results[parameter][c1][c2] = {'f1': f1, 'acc':acc}
    with open('results_individual_cnn.pkl', 'wb') as f:
        pickle.dump(results, f)
    print(c1, results[parameter][c1])

unk
(12115, 300)


100%|██████████| 629/629 [00:00<00:00, 1538.39it/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
  0%|          | 0/20 [00:00<?, ?it/s]

1.089381456375122
1.1479748487472534
1.144258737564087
1.072033405303955
1.093080759048462
1.0538885593414307
1.1238819360733032
1.022478699684143
1.1259276866912842
0.9815852642059326
1.2054498195648193
1.0640010833740234
1.2521427869796753
1.0156446695327759
0.9875408411026001
0.8740580677986145
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.56      0.63      0.60       180
         1.0       0.62      0.42      0.50       264
         2.0       0.37      0.50      0.43       185

    accuracy                           0.50       629
   macro avg       0.52      0.52      0.51       629
weighted avg       0.53      0.50      0.51       629

  Macro F1: 0.508
0
0.5077703529306449


  5%|▌         | 1/20 [00:03<01:09,  3.68s/it]

1.0386199951171875
0.9275733828544617
1.0297380685806274
1.0071437358856201
0.8545030951499939
0.9822649359703064
0.965736985206604
0.9728171825408936
0.8885053396224976
0.7765429019927979
0.8304867148399353
0.7639620304107666
0.7529614567756653
0.7230051159858704
0.8028945326805115
0.823376476764679
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.53      0.83      0.65       180
         1.0       0.65      0.70      0.67       264
         2.0       0.65      0.23      0.34       185

    accuracy                           0.60       629
   macro avg       0.61      0.59      0.55       629
weighted avg       0.62      0.60      0.57       629

  Macro F1: 0.554
0.5077703529306449
0.5544666005179755


...

              precision    recall  f1-score   support

         0.0       0.68      0.28      0.39        54
         1.0       0.66      0.53      0.59       131
         2.0       0.40      0.68      0.51        85

    accuracy                           0.53       270
   macro avg       0.58      0.50      0.50       270
weighted avg       0.59      0.53      0.52       270



100%|██████████| 232/232 [00:00<00:00, 1443.72it/s]


              precision    recall  f1-score   support

         0.0       0.68      0.55      0.61        51
         1.0       0.70      0.50      0.58       144
         2.0       0.20      0.49      0.29        37

    accuracy                           0.51       232
   macro avg       0.53      0.51      0.49       232
weighted avg       0.62      0.51      0.54       232



100%|██████████| 124/124 [00:00<00:00, 1438.18it/s]


              precision    recall  f1-score   support

         0.0       0.69      0.56      0.62        39
         1.0       0.33      0.65      0.43        20
         2.0       0.67      0.54      0.60        65

    accuracy                           0.56       124
   macro avg       0.56      0.58      0.55       124
weighted avg       0.62      0.56      0.58       124

African {'Caucasian': {'f1': 0.5521955849345611, 'acc': 0.548489666136725}, 'Hispanic': {'f1': 0.49617370099968866, 'acc': 0.5259259259259259}, 'Indian': {'f1': 0.4932305345303057, 'acc': 0.5086206896551724}, 'African': {'f1': 0.5504474138276955, 'acc': 0.5645161290322581}}


In [46]:
class_names = ['Caucasian', 'Hispanic', 'Indian', 'African']
parameter = 'Race'
# class_names = ['USA', 'Brazil', 'India', 'Spain', 'Canada', 'Italy']
# parameter = 'Country'
# class_names = ['Christian','Other', 'Nonreligious', 'Hindu']
# parameter = 'Religion'
# class_names = ['Male','Female', 'Prefer not to disclose']
# parameter = 'Gender'
df = pd.read_csv('data/annotators_race.csv')
# df = pd.read_csv('data/annotators_final.csv')
# df = pd.read_csv('data/annotators_religion.csv')
# df = pd.read_csv('data/annotators_gender.csv')
# print(df['Gender'])
df = df[df[parameter].isin(class_names)]
df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname=parameter,
                                         frac_train=0.8, frac_val=0.1, frac_test=0.1,
                                         random_state=42)
# len(df_train), len(df_val), len(df_test)

try:
    with open('results_combined_cnn.pkl', 'rb') as f:
        results = pickle.load(f)
except:
    results = {}
    with open('results_combined_cnn.pkl', 'wb') as f:
        pickle.dump(results, f)
print(results)
if parameter not in results: results[parameter] = {}

{'Gender': {'Male': {'f1': 0.6000808268413902, 'acc': 0.6210526315789474}, 'Female': {'f1': 0.5834714895284338, 'acc': 0.5908771929824561}, 'Prefer not to disclose': {'f1': 0.6098710391183818, 'acc': 0.6287719298245614}, 'full': {'f1': 0.6193970896373923, 'acc': 0.6252631578947369}}, 'Religion': {'Christian': {'f1': 0.6283349218971089, 'acc': 0.6638115631691649}, 'Other': {'f1': 0.6350541231355317, 'acc': 0.6502498215560314}, 'Nonreligious': {'f1': 0.6065857764161534, 'acc': 0.6167023554603854}, 'Hindu': {'f1': 0.6155309406838244, 'acc': 0.6581013561741613}, 'full': {'f1': 0.6142789626409518, 'acc': 0.6409707351891506}}, 'Country': {'USA': {'f1': 0.5956362256570017, 'acc': 0.6157407407407407}, 'Brazil': {'f1': 0.5987399961537893, 'acc': 0.6041666666666666}, 'India': {'f1': 0.6328762668830547, 'acc': 0.6458333333333334}, 'Spain': {'f1': 0.6036103283712723, 'acc': 0.6373456790123457}, 'Canada': {'f1': 0.617792127686582, 'acc': 0.6311728395061729}, 'Italy': {'f1': 0.6184201114480106, 'acc

In [48]:
for c1 in class_names:
    train_df = df_train[~df_train[parameter].isin([c1])]
    val_df = df_val

    vocab_own = Vocab_own(train_df, wordvec_model)
    vocab_own.create_vocab()

    X_train=encodeData(train_df,vocab_own)
    train_dataloader = combine_features(X_train,True)

    X_val=encodeData(val_df,vocab_own)
    val_dataloader = combine_features(X_val)

    model = CNN_GRU(vocab_own.embeddings).to(device)
    model = train(train_dataloader, val_dataloader, model, epochs = 20)

    test_df = df_test
    X_test=encodeData(test_df,vocab_own)
    test_dataloader = combine_features(X_test)
    f1, ypreds, ytest = evaluate(test_dataloader, model)
    acc = accuracy_score(ytest, ypreds)
    results[parameter][c1] = {'f1': f1, 'acc':acc}
    with open('results_combined_cnn.pkl', 'wb') as f:
        pickle.dump(results, f)
    print(c1, results[parameter][c1])

train_df = df_train
val_df = df_val

vocab_own = Vocab_own(train_df, wordvec_model)
vocab_own.create_vocab()

X_train=encodeData(train_df,vocab_own)
train_dataloader = combine_features(X_train,True)

X_val=encodeData(val_df,vocab_own)
val_dataloader = combine_features(X_val)

model = CNN_GRU(vocab_own.embeddings).to(device)
model = train(train_dataloader, val_dataloader, model, epochs = 20)
test_df = df_test
X_test=encodeData(test_df,vocab_own)
test_dataloader = combine_features(X_test)
f1, ypreds, ytest = evaluate(test_dataloader, model)
acc = accuracy_score(ytest, ypreds)
results[parameter]['full'] = {'f1': f1, 'acc':acc}
with open('results_combined_cnn.pkl', 'wb') as f:
    pickle.dump(results, f)
print(c1, results[parameter]['full'])

unk
(12180, 300)



100%|██████████| 5009/5009 [00:03<00:00, 1455.99it/s]

100%|██████████| 1255/1255 [00:00<00:00, 1449.96it/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))

  0%|          | 0/20 [00:00<?, ?it/s]

1.103907585144043
1.126105785369873
1.082350730895996
1.1037487983703613
1.1475814580917358
1.0227572917938232
1.0794851779937744
1.06068754196167
1.0553054809570312
1.0089900493621826
0.9567774534225464
0.9946205019950867
1.0091748237609863
0.8446937203407288
1.0067031383514404
0.91108238697052
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.74      0.48      0.58       365
         1.0       0.55      0.88      0.68       545
         2.0       0.44      0.20      0.28       345

    accuracy                           0.58      1255
   macro avg       0.58      0.52      0.51      1255
weighted avg       0.58      0.58      0.54      1255

  Macro F1: 0.512
0
0.512263034436346



  5%|▌         | 1/20 [00:03<01:11,  3.79s/it]

0.7588187456130981
1.0829429626464844
0.7544856071472168
0.8055716753005981
1.006447672843933
0.7851685881614685
0.8103593587875366
0.7871085405349731
0.7906843423843384
1.0758417844772339
1.128636121749878
0.9233438968658447
1.0312469005584717
0.6543855667114258
0.9404736757278442
1.1298673152923584
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.70      0.64      0.67       365
         1.0       0.63      0.77      0.69       545
         2.0       0.48      0.35      0.40       345

    accuracy                           0.62      1255
   macro avg       0.60      0.59      0.59      1255
weighted avg       0.61      0.62      0.61      1255

  Macro F1: 0.588
0.512263034436346
0.5881672833742551


...

0.1815747618675232
0.14355820417404175
0.3857755959033966
0.020160309970378876
0.079584039747715
0.04435284435749054
0.37143629789352417
0.061277054250240326
0.026872610673308372
0.1263299286365509
0.05038023367524147
0.04447396099567413
0.1407437026500702
0.09476610273122787
0.18989789485931396
0.10670097917318344
0.08394131809473038
0.09122753143310547
0.05297496169805527
0.09604787081480026
0.15465468168258667
0.24663284420967102
0.22060957551002502
0.07700928300619125
0.08634024858474731
0.005827695596963167
0.09200440347194672
0.2574334740638733
0.03239400312304497
0.07440434396266937
0.014356561936438084
0.04773649945855141
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.57      0.64      0.60       365
         1.0       0.66      0.62      0.64       545
         2.0       0.42      0.41      0.41       345

    accuracy                           0.57      1255
   macro avg       0.55      0.55      0.55      1255
weighted av


 95%|█████████▌| 19/20 [02:21<00:07,  7.41s/it]

0.05117788538336754
0.029102426022291183
0.19657397270202637
0.07875484228134155
0.20649893581867218
0.04504506662487984
0.29963186383247375
0.02661726623773575
0.07806277275085449
0.04267130419611931
0.20705804228782654
0.032625384628772736
0.052970465272665024
0.024178270250558853
0.1120414137840271
0.04182031378149986
0.432709664106369
0.07603800296783447
0.30745384097099304
0.05409960448741913
0.10529809445142746
0.16340193152427673
0.09549234062433243
0.33805567026138306
0.20324395596981049
0.06705483794212341
0.1832243651151657
0.024380605667829514
0.18204942345619202
0.04327741637825966
0.30789461731910706
0.19299118220806122
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.61      0.60      0.60       365
         1.0       0.64      0.66      0.65       545
         2.0       0.44      0.43      0.44       345

    accuracy                           0.58      1255
   macro avg       0.57      0.56      0.56      1255
weighted


100%|██████████| 20/20 [02:28<00:00,  7.43s/it]

100%|██████████| 1255/1255 [00:00<00:00, 1406.34it/s]


              precision    recall  f1-score   support

         0.0       0.67      0.71      0.69       346
         1.0       0.67      0.76      0.71       544
         2.0       0.54      0.41      0.47       365

    accuracy                           0.64      1255
   macro avg       0.63      0.62      0.62      1255
weighted avg       0.63      0.64      0.63      1255

African {'f1': 0.6204438701985747, 'acc': 0.6406374501992032}


In [50]:
results

{'Country': {'Brazil': {'acc': 0.6041666666666666, 'f1': 0.5987399961537893},
  'Canada': {'acc': 0.6311728395061729, 'f1': 0.617792127686582},
  'India': {'acc': 0.6458333333333334, 'f1': 0.6328762668830547},
  'Italy': {'acc': 0.6458333333333334, 'f1': 0.6184201114480106},
  'Spain': {'acc': 0.6373456790123457, 'f1': 0.6036103283712723},
  'USA': {'acc': 0.6157407407407407, 'f1': 0.5956362256570017},
  'full': {'acc': 0.6520061728395061, 'f1': 0.6387210139439162}},
 'Gender': {'Female': {'acc': 0.5908771929824561, 'f1': 0.5834714895284338},
  'Male': {'acc': 0.6210526315789474, 'f1': 0.6000808268413902},
  'Prefer not to disclose': {'acc': 0.6287719298245614,
   'f1': 0.6098710391183818},
  'full': {'acc': 0.6252631578947369, 'f1': 0.6193970896373923}},
 'Race': {'African': {'acc': 0.6374501992031872, 'f1': 0.6055418329028067},
  'Caucasian': {'acc': 0.6039840637450199, 'f1': 0.602005549501086},
  'Hispanic': {'acc': 0.6446215139442231, 'f1': 0.6294359817627183},
  'Indian': {'acc': 